<a href="https://colab.research.google.com/github/tayfununal/nku_lesson/blob/main/continuous_last_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import categorical_crossentropy, mse, CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import concatenate

In [ ]:
# Loading original data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshaping the array to 4-dims so that it can work with the Keras API
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# Making sure that the values are float so that we can get decimal points after division
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalizing the RGB codes by dividing it to the max RGB value.
X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape)
print('Number of images in X_train', X_train.shape[0])
print('Number of images in X_test', X_test.shape[0])

In [ ]:
# Adding epsilon
tf.random.set_seed(8)
epsilon = tf.random.normal((60000,28,28,1), mean=0, stddev= 0.00001)
epsilon_test = tf.random.normal((10000,28,28,1), mean=0, stddev= 0.00001)

X_train_with_epsilon = tf.math.add(X_train, epsilon)
X_test_with_epsilon = tf.math.add(X_test, epsilon_test)

print('X_train shape:', X_train_with_epsilon.shape)
print('Number of images in X_train', X_train_with_epsilon.shape[0])
print('Number of images in X_test', X_test_with_epsilon.shape[0])

In [ ]:
# Making y_train and y_test to categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
np.random.seed(8)
tf.random.set_seed(8)

feature_input = Input(shape=(28,28))

hidden = Flatten()(feature_input)
hidden = Dense(512, activation="relu", name="one")(hidden)
hidden = Dense(512, activation="relu")(hidden)
hidden = Dense(256, activation="relu")(hidden)
hidden = Dense(256, activation="relu")(hidden)
hidden = Dense(128, activation="relu")(hidden)

output = Dense(10, activation='softmax')(hidden)

model = Model(inputs=feature_input , outputs=output)
model.summary()

In [ ]:
batch_size = 60000
lamdas = 0.7
import keras.backend as K
class CustomMSE(tf.keras.losses.Loss):

    def __init__(self, regularization_factor=0.1, name="custom_mse"):
        super().__init__(name=name)
        self.regularization_factor = regularization_factor

    def call(self, y_true, y_pred):
        # custom_cross_entropy_loss = -1/batch_size * tf.reduce_sum(y_true * tf.math.log(y_pred + 10e-15))
        cross_entropy_loss = CategoricalCrossentropy()(y_true, y_pred)
        reg = lamdas * (1/batch_size) * tf.reduce_sum(abs(y_pred - model(X_train_with_epsilon)))

        return cross_entropy_loss + reg

In [ ]:
# Culculating Custom Loss
model.compile(optimizer="Adam", loss=CustomMSE(), metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=60000)

In [ ]:
;# Culculating Custom Loss
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=100, verbose=1, batch_size=60000)